> # **Meta Estimator in sk-lego Threshold**
>
> ## Use for changing model line (LogisticRegres (sionsigmoid))  position 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import make_classification
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score, make_scorer

from sklego.meta import Thresholder
X,y = make_classification(n_samples=2000,n_features=2,n_redundant=0,random_state=21)
plt.scatter(x=X[:,0], y=X[:,1] , c=y, s=40, edgecolors='silver') 

In [ ]:
m1 = Thresholder(LogisticRegression(solver='lbfgs'), threshold = 0.1).fit(X,y)  # This solver is default
m2 = Thresholder(LogisticRegression(solver='lbfgs'), threshold = 0.9).fit(X,y)

plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.scatter (X[:, 0], X[:, 1], c=y, s=5)
plt.title("original data")
plt.subplot(132)
plt.scatter(X[:, 0], X[:, 1], c=m1.predict(X), s=5)
plt.title("threshold=0.1")
plt.subplot(133)
plt.scatter (X[:, 0], X[:, 1], c=m2.predict(X), s=5)
plt.title("threshold=0.9");

In [ ]:
pipe = Pipeline([('model', Thresholder(LogisticRegression(solver='lbfgs'), threshold = 0.1))])

mod = GridSearchCV(estimator=pipe,                 
                   param_grid = {'model__threshold':np.linspace(0.1, 0.9, 50)},  # Start with 0.1-0.9  within 50 points inside
                   scoring = {'precision': make_scorer(precision_score),
                              'recall': make_scorer(recall_score),
                              'accuracy': make_scorer(accuracy_score)},
                   refit = 'precision',
                   cv = 5)
mod.fit(X,y)

In [ ]:
mod.predict(X)

In [ ]:
df_results = pd.DataFrame(mod.cv_results_)

In [ ]:
# Plotting the results
plt.figure(figsize=(12, 4))
plt.plot(df_results["param_model__threshold"], df_results["mean_test_precision"], label="precision")
plt.plot(df_results["param_model__threshold"], df_results["mean_test_recall"], label="recall") 
plt.plot(df_results["param_model__threshold"], df_results["mean_test_accuracy"], label="accuracy")
plt.legend()
plt.title("performance")
plt.plot()

### *After this plotting We find the best position for threshold **(where all lines joint)***